# Testing for UNION - using all spindles, training with all data
confidence > 0.5

fistly define the UUID of the experiment to use

In [1]:
experimentId="6e8f728f-81ce-4048-8acf-f8d1ffc89a39"

-------------------------------------------------------

In [2]:
#external libraries
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.colors as clt
import plotly
import plotly.subplots as sb
import plotly.express as px
import plotly.graph_objects as go
import dotenv
import pandas as pd
import scipy.fft as fft
import scipy.signal as sg
import scipy.io as sio
import pickle as pkl
import xgboost as xgb
import time
import sklearn.metrics as skm

#project library
from spinco import *

#environment variables
dotenv.load_dotenv('lab.env')

#project variables
datapath=os.environ['DATAPATH']
cognipath=datapath+"\\COGNITION"
dreamspath=datapath+"\\DREAMS"
masspath=datapath+"\\MASS"


## load mass

In [3]:
samplerate=200  #Should rethink this

In [4]:
annotations, signalsMetadata = loadMASSSpindles(masspath,forceSamplerate=samplerate)

In [5]:
#consider to include this in a function
minDuration=0.3
maxDuration=2.9
annotations=annotations[annotations.type=='spindle']
annotations=annotations[annotations.duration>minDuration]
annotations=annotations[annotations.duration<maxDuration]
annotations=annotations.reset_index(drop=True)

In [6]:
#check with EDA results (not needed)
print(1-len(annotations)/33458)

0.011297746428357902


In [8]:
annotations.head()

,type,expert,subjectId,labelerId,startTime,duration,samplerate,stopTime,startInd,stopInd
0,spindle,E1,0001,0001,888.327805,0.640579,200,888.968384,177666,177794
1,spindle,E1,0001,0001,905.758061,0.578094,200,906.336155,181152,181267
2,spindle,E1,0001,0001,917.731574,0.847603,200,918.579177,183546,183716
3,spindle,E1,0001,0001,922.078189,0.878845,200,922.957034,184416,184591
4,spindle,E1,0001,0001,939.055445,0.757767,200,939.813212,187811,187963


In [9]:
signalsMetadata.head()

,subjectId,file,channel,duration,samplerate,isOriginalSamplerate,database
0,0001,MASS_0001.pkl,C3-CLE,28956.0,200,False,MASS
1,0002,MASS_0002.pkl,C3-CLE,35016.0,200,False,MASS
2,0003,MASS_0003.pkl,C3-CLE,36760.0,200,False,MASS
3,0004,MASS_0004.pkl,C3-CLE,28004.0,200,False,MASS
4,0005,MASS_0005.pkl,C3-CLE,31244.0,200,False,MASS


## Load experiment results

In [10]:
experimentModels, featureSelection = loadExperiment(experimentId,datapath)

In [11]:
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate
0,0001,E1,[0001],"[0003, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0017, 0002, 0006]",0001,9e4dd9bb-a7d9-4ee8-a762-d656d8308d91,0.018639
1,0001,E1,[0001],"[0001, 0003, 0005, 0007, 0009, 0010, 0011, 001...","[0006, 0013, 0019]",0002,5e1bcb8a-f879-4f42-ae21-0e1ae715095f,0.020055
2,0001,E1,[0001],"[0002, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0013, 0006, 0001]",0003,2ecef3ee-0879-42d7-bcf0-ecfa0e3e601a,0.021075
3,0001,E1,[0001],"[0001, 0002, 0003, 0006, 0007, 0009, 0010, 001...","[0018, 0014, 0012]",0005,52b0d6a8-7f01-4694-85d4-14ef17a025b9,0.018331
4,0001,E1,[0001],"[0003, 0007, 0009, 0010, 0011, 0012, 0013, 001...","[0005, 0001, 0002]",0006,32c9793a-2115-4e77-8b6f-e8f71d09cecb,0.019222
5,0001,E1,[0001],"[0001, 0002, 0003, 0010, 0011, 0012, 0013, 001...","[0005, 0006, 0009]",0007,241d9889-61ff-47f7-97c2-f502ae3dba38,0.019625
6,0001,E1,[0001],"[0001, 0003, 0005, 0006, 0007, 0010, 0011, 001...","[0002, 0017, 0013]",0009,f6a46037-1f32-40b5-a6ff-cd293b0e4203,0.017824
7,0001,E1,[0001],"[0002, 0003, 0005, 0006, 0007, 0009, 0012, 001...","[0017, 0001, 0011]",0010,6cfc1f74-3653-47eb-a166-f59fefa13825,0.017542
8,0001,E1,[0001],"[0001, 0003, 0005, 0006, 0009, 0010, 0012, 001...","[0002, 0018, 0007]",0011,68c5483d-9072-4478-8a87-a7af3324f690,0.015442
9,0001,E1,[0001],"[0002, 0003, 0005, 0006, 0009, 0011, 0013, 001...","[0001, 0007, 0010]",0012,349fe35c-b749-431d-8e8c-c288b150e998,0.016460


In [12]:
#we show the difference in class inbalance for the annotation criteria considered
experimentModels[['criteriumName','spindleTimeRate']].groupby('criteriumName').describe()

spindleTimeRate                                          \
                        count      mean       std       min       25%   
criteriumName                                                           
E1                       15.0  0.018463  0.001834  0.015442  0.017419   
E2                       15.0  0.058566  0.004500  0.052646  0.055195   
union                    15.0  0.059793  0.004559  0.054024  0.056196   

                                             
                    50%       75%       max  
criteriumName                                
E1             0.018331  0.019424  0.022439  
E2             0.057005  0.061849  0.067263  
union          0.058468  0.062907  0.068772

In [13]:
featureSelection

,characteristic,bandName,window
21,hjortActivity,sigma,0.5
66,petrosian,broadband,0.5
22,hjortActivity,theta,0.5
79,relativePower,beta1,0.5
29,hjortComplexity,sigma,0.5


## Hyperparameter definition
this should come from a previous evaluation notebook

In [14]:
hyperClose=0.25
hyperDuration=0.3
hyperThres=0.4
hyperDepth=40
hyperConfidence=0.5

## Testing with E2 criterium
we test the optimal points for the prediction threshold and number of boost iterations in the different validation groups

In [15]:
experimentModels=experimentModels[experimentModels.criteriumName=='union'].reset_index(drop=True)
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate
0,0003,union,"[0001, 0002]","[0003, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0017, 0002, 0006]",0001,f3ec491b-0006-4caa-ae15-ae896cb2d67e,0.058468
1,0003,union,"[0001, 0002]","[0001, 0003, 0005, 0007, 0009, 0010, 0011, 001...","[0006, 0013, 0019]",0002,6491a608-d9e0-4af8-bfcb-4213b7921942,0.062867
2,0003,union,"[0001, 0002]","[0002, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0013, 0006, 0001]",0003,257f50f6-47f8-43cb-bd89-084609bb54f4,0.064768
3,0003,union,"[0001, 0002]","[0001, 0002, 0003, 0006, 0007, 0009, 0010, 001...","[0018, 0014, 0012]",0005,1531e4ee-35ce-4d1c-a1f9-c3c93a7adb13,0.062913
4,0003,union,"[0001, 0002]","[0003, 0007, 0009, 0010, 0011, 0012, 0013, 001...","[0005, 0001, 0002]",0006,ea67e468-eb31-4b91-9973-01efd704fdfd,0.060454
5,0003,union,"[0001, 0002]","[0001, 0002, 0003, 0010, 0011, 0012, 0013, 001...","[0005, 0006, 0009]",0007,36af1678-f8a9-4522-bf98-7726af8393a9,0.062901
6,0003,union,"[0001, 0002]","[0001, 0003, 0005, 0006, 0007, 0010, 0011, 001...","[0002, 0017, 0013]",0009,da8325b6-379e-496f-acb8-3c105a943c56,0.057417
7,0003,union,"[0001, 0002]","[0002, 0003, 0005, 0006, 0007, 0009, 0012, 001...","[0017, 0001, 0011]",0010,a832e8e9-3213-4caf-b34f-cdd51a27167b,0.054024
8,0003,union,"[0001, 0002]","[0001, 0003, 0005, 0006, 0009, 0010, 0012, 001...","[0002, 0018, 0007]",0011,8e0df4bf-4d23-4aa8-aa60-b26ac4bc4dbf,0.055899
9,0003,union,"[0001, 0002]","[0002, 0003, 0005, 0006, 0009, 0011, 0013, 001...","[0001, 0007, 0010]",0012,a4b03b48-4c81-4340-b92a-12f602f553a2,0.054366


In [16]:
#threshold for the metric by event
masterIoU=0.2

In [17]:
def getIou(coordA,coordB):
    if (coordA[1]<coordB[0])|(coordB[1]<coordA[0]): #NOT INTERSECTION
        iou=0
    else:   #INTERSECTION
        inter=np.min((coordA[1]-coordB[0],coordB[1]-coordA[0]))
        union=np.max((coordA[1]-coordB[0],coordB[1]-coordA[0]))
        iou=inter/union
    return iou

def annotationPairToMetrics(annotations,detections,thresIoU=0.2):
    #get the coords
    gtCoords=zip(annotations.startInd,annotations.stopInd)
    outCoords=zip(detections.startInd,detections.stopInd)
    #calculate the iou vector
    iouVector=np.array(list(itt.starmap(getIou,itt.product(gtCoords,outCoords))))
    #reshape to a matrix
    iouMatrix=iouVector.reshape(len(annotations),len(detections))
    #binarize
    binarized=iouMatrix>thresIoU
    #calculateF1
    outF1=(np.sum(np.max(binarized,axis=0))+np.sum(np.max(binarized,axis=1)))/(len(annotations)+len(detections))
    recall=np.sum(np.max(binarized,axis=1))/len(annotations)
    precision=np.sum(np.max(binarized,axis=0))/len(detections)
    return outF1,recall,precision

In [20]:
rawF1s=[]
rawPrecisions=[]
rawRecalls=[]

f1s=[]
precisions=[]
recalls=[]

eventF1s=[]
eventPrecisions=[]
eventRecalls=[]

checks=[]

for ind,row in experimentModels.iterrows():
    print('*************************')
    print(str(ind+1)+' of '+str(len(experimentModels)) )
    #load model
    model=loadBooster(row.modelId,experimentId,datapath)

    testSubjectId=row.test
    #Define annotations criterium
    usedAnnotations=annotations[annotations.labelerId.isin(row.labelerIdList)].reset_index(drop=True)
    #Load features and labels
    testFeatures=loadFeatureMatrix([testSubjectId],featureSelection,signalsMetadata,samplerate,datapath)
    testLabels=loadLabelsVector([testSubjectId],usedAnnotations,signalsMetadata,samplerate)

    #Predict
    testDMatrix=xgb.DMatrix(data=testFeatures)
    probabilities=model.predict(testDMatrix,iteration_range=(0,hyperDepth))
    rawLabels=probabilities>=hyperThres
    #Raw Metrics
    rawTp=np.sum(rawLabels*testLabels)
    rawFp=np.sum(rawLabels*(1-testLabels))
    rawTn=np.sum((1-rawLabels)*(1-testLabels))
    rawFn=np.sum((1-rawLabels)*testLabels)
    #Raw appends
    rawF1s.append(2*rawTp/(2*rawTp+rawFp+rawFn))
    rawPrecisions.append(rawTp/(rawTp+rawFp) )
    rawRecalls.append(rawTp/(rawTp+rawFn))
    #Process
    processedLabels=labelingProcess(rawLabels,hyperClose,hyperDuration,samplerate)
    #Detections with confidence
    detections=rawAndProcessedToConfidenceAnnotations(rawLabels,processedLabels,samplerate)
    #Confidence threshold
    detections=detections[detections.confidence>hyperConfidence].reset_index(drop=True)
    #recalculate processed labels
    processedLabels=excerptAnnotationsToLabels(detections,len(processedLabels))
    #Processed metrics
    tp=np.sum(processedLabels*testLabels)
    fp=np.sum(processedLabels*(1-testLabels))
    tn=np.sum((1-processedLabels)*(1-testLabels))
    fn=np.sum((1-processedLabels)*testLabels)
    #Processed appends
    f1s.append(2*tp/(2*tp+fp+fn))
    precisions.append(tp/(tp+fp))
    recalls.append(tp/(tp+fn))

    #By-event metrics
    gtAnnotations=labelVectorToAnnotations(testLabels,samplerate)   #<- or just filter the annotations
    f,r,p=annotationPairToMetrics(gtAnnotations,detections)
    
    #calculate metrics
    eventF1s.append(f)
    eventPrecisions.append(p)
    eventRecalls.append(r)

#include metrics in the dataframe
experimentModels['rawF1']=rawF1s
experimentModels['rawPrecision']=rawPrecisions
experimentModels['rawRecall']=rawRecalls

experimentModels['f1']=f1s
experimentModels['precision']=precisions
experimentModels['recall']=recalls

experimentModels['eventF1']=eventF1s
experimentModels['eventPrecision']=eventPrecisions
experimentModels['eventRecall']=eventRecalls


*************************
1 of 15
*************************
2 of 15
*************************
3 of 15
*************************
4 of 15
*************************
5 of 15
*************************
6 of 15
*************************
7 of 15
*************************
8 of 15
*************************
9 of 15
*************************
10 of 15
*************************
11 of 15
*************************
12 of 15
*************************
13 of 15
*************************
14 of 15
*************************
15 of 15


In [22]:
dumpPickle('experimentModelsTest_union_confidence_temp.pkl',experimentModels)

In [23]:
experimentModels.columns

Index(['criteriumId', 'criteriumName', 'labelerIdList', 'train', 'val', 'test',
       'modelId', 'spindleTimeRate', 'rawF1', 'rawPrecision', 'rawRecall',
       'f1', 'precision', 'recall', 'eventF1', 'eventPrecision',
       'eventRecall'],
      dtype='object')

In [24]:
fig=px.scatter(experimentModels,x='rawF1',y='f1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['rawF1'], y=experimentModels['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [25]:
fig=px.scatter(experimentModels,x='rawF1',y='eventF1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['rawF1'], y=experimentModels['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [26]:
fig=px.scatter(experimentModels,x='eventF1',y='eventPrecision',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [27]:
fig=px.scatter(experimentModels,x='eventF1',y='eventRecall',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [28]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test').describe(percentiles=[0.5])

eventF1                                             eventPrecision  \
       count      mean std       min       50%       max          count   
test                                                                      
0001     1.0  0.574744 NaN  0.574744  0.574744  0.574744            1.0   
0002     1.0  0.843381 NaN  0.843381  0.843381  0.843381            1.0   
0003     1.0  0.729570 NaN  0.729570  0.729570  0.729570            1.0   
0005     1.0  0.747158 NaN  0.747158  0.747158  0.747158            1.0   
0006     1.0  0.680851 NaN  0.680851  0.680851  0.680851            1.0   
0007     1.0  0.689249 NaN  0.689249  0.689249  0.689249            1.0   
0009     1.0  0.755900 NaN  0.755900  0.755900  0.755900            1.0   
0010     1.0  0.771912 NaN  0.771912  0.771912  0.771912            1.0   
0011     1.0  0.785305 NaN  0.785305  0.785305  0.785305            1.0   
0012     1.0  0.714697 NaN  0.714697  0.714697  0.714697            1.0   
0013     1.0  0.710392 NaN  0.710392  0.710392  0.710392            1.0   
0014     1.0  0.622266 NaN  0.622266  0.622266  0.622266            1.0   
0017     1.0  0.766739 NaN  0.766739  0.766739  0.766739            1.0   
0018     1.0  0.768318 NaN  0.768318  0.768318  0.768318            1.0   
0019     1.0  0.715889 NaN  0.715889  0.715889  0.715889            1.0   

                                                 eventRecall                \
          mean std       min       50%       max       count      mean std   
test                                                                         
0001  0.879121 NaN  0.879121  0.879121  0.879121         1.0  0.426929 NaN   
0002  0.792177 NaN  0.792177  0.792177  0.792177         1.0  0.901943 NaN   
0003  0.741880 NaN  0.741880  0.741880  0.741880         1.0  0.717608 NaN   
0005  0.632439 NaN  0.632439  0.632439  0.632439         1.0  0.912532 NaN   
0006  0.794872 NaN  0.794872  0.794872  0.794872         1.0  0.595438 NaN   
0007  0.545104 NaN  0.545104  0.545104  0.545104         1.0  0.937689 NaN   
0009  0.873839 NaN  0.873839  0.873839  0.873839         1.0  0.664875 NaN   
0010  0.748316 NaN  0.748316  0.748316  0.748316         1.0  0.797214 NaN   
0011  0.671495 NaN  0.671495  0.671495  0.671495         1.0  0.945431 NaN   
0012  0.591534 NaN  0.591534  0.591534  0.591534         1.0  0.903487 NaN   
0013  0.560466 NaN  0.560466  0.560466  0.560466         1.0  0.970077 NaN   
0014  0.453246 NaN  0.453246  0.453246  0.453246         1.0  0.986445 NaN   
0017  0.795699 NaN  0.795699  0.795699  0.795699         1.0  0.739783 NaN   
0018  0.651796 NaN  0.651796  0.651796  0.651796         1.0  0.935447 NaN   
0019  0.566185 NaN  0.566185  0.566185  0.566185         1.0  0.971510 NaN   

                                    
           min       50%       max  
test                                
0001  0.426929  0.426929  0.426929  
0002  0.901943  0.901943  0.901943  
0003  0.717608  0.717608  0.717608  
0005  0.912532  0.912532  0.912532  
0006  0.595438  0.595438  0.595438  
0007  0.937689  0.937689  0.937689  
0009  0.664875  0.664875  0.664875  
0010  0.797214  0.797214  0.797214  
0011  0.945431  0.945431  0.945431  
0012  0.903487  0.903487  0.903487  
0013  0.970077  0.970077  0.970077  
0014  0.986445  0.986445  0.986445  
0017  0.739783  0.739783  0.739783  
0018  0.935447  0.935447  0.935447  
0019  0.971510  0.971510  0.971510

In [29]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean()

,test,eventF1,eventPrecision,eventRecall
0,0001,0.574744,0.879121,0.426929
1,0002,0.843381,0.792177,0.901943
2,0003,0.729570,0.741880,0.717608
3,0005,0.747158,0.632439,0.912532
4,0006,0.680851,0.794872,0.595438
5,0007,0.689249,0.545104,0.937689
6,0009,0.755900,0.873839,0.664875
7,0010,0.771912,0.748316,0.797214
8,0011,0.785305,0.671495,0.945431
9,0012,0.714697,0.591534,0.903487


In [30]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().mean()

test              6.668000e+54
eventF1           7.250915e-01
eventPrecision    6.865446e-01
eventRecall       8.270938e-01
dtype: float64

In [31]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().std()

C:\Users\Gurdiel\AppData\Local\Temp\ipykernel_9612\2931376690.py:1: FutureWarning:

The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



eventF1           0.066385
eventPrecision    0.128745
eventRecall       0.165620
dtype: float64

In [32]:
auxPrecision=pd.DataFrame({
    'metric':'event precision',
    'value':experimentModels.eventPrecision,
    'event F1':experimentModels.eventF1
})

auxRecall=pd.DataFrame({
    'metric':'event recall',
    'value':experimentModels.eventRecall,
    'event F1':experimentModels.eventF1
})
visualTradeoff=pd.concat((auxPrecision,auxRecall))

In [33]:
fig=px.scatter(visualTradeoff,x='event F1',y='value',color='metric', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

## Restrict to only N2 stage

In [34]:
experimentModels_N2=experimentModels.copy()
experimentModels_N2.head()

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate,rawF1,rawPrecision,rawRecall,f1,precision,recall,eventF1,eventPrecision,eventRecall
0,0003,union,"[0001, 0002]","[0003, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0017, 0002, 0006]",0001,f3ec491b-0006-4caa-ae15-ae896cb2d67e,0.058468,0.393555,0.897916,0.252004,0.423763,0.903647,0.276779,0.574744,0.879121,0.426929
1,0003,union,"[0001, 0002]","[0001, 0003, 0005, 0007, 0009, 0010, 0011, 001...","[0006, 0013, 0019]",0002,6491a608-d9e0-4af8-bfcb-4213b7921942,0.062867,0.715863,0.812713,0.639638,0.747613,0.814099,0.691167,0.843381,0.792177,0.901943
2,0003,union,"[0001, 0002]","[0002, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0013, 0006, 0001]",0003,257f50f6-47f8-43cb-bd89-084609bb54f4,0.064768,0.600816,0.751701,0.500377,0.629942,0.760062,0.537863,0.729570,0.741880,0.717608
3,0003,union,"[0001, 0002]","[0001, 0002, 0003, 0006, 0007, 0009, 0010, 001...","[0018, 0014, 0012]",0005,1531e4ee-35ce-4d1c-a1f9-c3c93a7adb13,0.062913,0.677448,0.640797,0.718545,0.698105,0.636924,0.772288,0.747158,0.632439,0.912532
4,0003,union,"[0001, 0002]","[0003, 0007, 0009, 0010, 0011, 0012, 0013, 001...","[0005, 0001, 0002]",0006,ea67e468-eb31-4b91-9973-01efd704fdfd,0.060454,0.529312,0.772381,0.402610,0.559360,0.782027,0.435391,0.680851,0.794872,0.595438


In [35]:
annotationsN2=pd.read_csv(datapath+'/MASS/annotations/spindlesFilteredN2.csv')
annotationsN2['samplerate']=samplerate
annotationsN2['subjectId']=annotationsN2.apply(
    lambda row: str(row.subjectId).zfill(4),axis=1)
annotationsN2['labelerId']=annotationsN2.apply(
    lambda row: str(row.labelerId).zfill(4),axis=1)
annotationsN2['stopTime']=annotationsN2.apply(
    lambda row: row.startTime+row.duration , axis=1)
annotationsN2['startInd']=annotationsN2.apply(
    lambda row: seconds2index(row.startTime,row.samplerate) , axis=1)
annotationsN2['stopInd']=annotationsN2.apply(
    lambda row: seconds2index(row.stopTime,row.samplerate) , axis=1)

In [36]:
## load hypnograms
stagesAnnotations=pd.read_csv(datapath+"\MASS\stages\stages.csv")
stagesAnnotations['samplerate']=samplerate
stagesAnnotations['subjectId']=stagesAnnotations.apply(
    lambda row: str(row.subjectId).zfill(4),axis=1)
stagesAnnotations['stopTime']=stagesAnnotations.apply(
    lambda row: row.startTime+row.duration , axis=1)
stagesAnnotations['startInd']=stagesAnnotations.apply(
    lambda row: seconds2index(row.startTime,row.samplerate) , axis=1)
stagesAnnotations['stopInd']=stagesAnnotations.apply(
    lambda row: seconds2index(row.stopTime,row.samplerate) , axis=1)
stagesAnnotations.head(5)

hypnograms={}
for ind, row in signalsMetadata.iterrows():
    subjectId=row.subjectId
    thisStages=stagesAnnotations[stagesAnnotations.subjectId==subjectId]
    excerptDimension=int(row.duration*row.samplerate)
    thisHypnogram=np.ones((excerptDimension,))*np.nan
    for ind_stg, row_stg in thisStages.iterrows():
        thisHypnogram[row_stg.startInd:row_stg.stopInd]=row_stg.value
    hypnograms[subjectId]=thisHypnogram   

In [37]:
rawF1s=[]
rawPrecisions=[]
rawRecalls=[]

f1s=[]
precisions=[]
recalls=[]

eventF1s=[]
eventPrecisions=[]
eventRecalls=[]

checks=[]

for ind,row in experimentModels_N2.iterrows():
    print('*************************')
    print(str(ind+1)+' of '+str(len(experimentModels_N2)) )
    #load model
    model=loadBooster(row.modelId,experimentId,datapath)

    testSubjectId=row.test
    testHypnogram=hypnograms[testSubjectId]
    testN2Mask=testHypnogram==2
    #Define annotations criterium
    usedAnnotations=annotationsN2[annotationsN2.labelerId.isin(row.labelerIdList)].reset_index(drop=True)
    #Load features and labels
    testFeatures=loadFeatureMatrix([testSubjectId],featureSelection,signalsMetadata,samplerate,datapath)
    testLabels=loadLabelsVector([testSubjectId],usedAnnotations,signalsMetadata,samplerate)

    #Predict
    testDMatrix=xgb.DMatrix(data=testFeatures)
    probabilities=model.predict(testDMatrix,iteration_range=(0,hyperDepth))
    rawLabels=probabilities>=hyperThres

    #Apply mask ---------------------------------->
    rawLabels=rawLabels*testN2Mask
    #<---------------------------------------------

    #Raw Metrics
    rawTp=np.sum(rawLabels*testLabels)
    rawFp=np.sum(rawLabels*(1-testLabels))
    rawTn=np.sum((1-rawLabels)*(1-testLabels))
    rawFn=np.sum((1-rawLabels)*testLabels)
    #Raw appends
    rawF1s.append(2*rawTp/(2*rawTp+rawFp+rawFn))
    rawPrecisions.append(rawTp/(rawTp+rawFp) )
    rawRecalls.append(rawTp/(rawTp+rawFn))
    #Process
    processedLabels=labelingProcess(rawLabels,hyperClose,hyperDuration,samplerate)
    #Detections with confidence
    detections=rawAndProcessedToConfidenceAnnotations(rawLabels,processedLabels,samplerate)
    #Confidence threshold
    detections=detections[detections.confidence>hyperConfidence].reset_index(drop=True)
    #recalculate processed labels
    processedLabels=excerptAnnotationsToLabels(detections,len(processedLabels))
    #Processed metrics
    tp=np.sum(processedLabels*testLabels)
    fp=np.sum(processedLabels*(1-testLabels))
    tn=np.sum((1-processedLabels)*(1-testLabels))
    fn=np.sum((1-processedLabels)*testLabels)
    #Processed appends
    f1s.append(2*tp/(2*tp+fp+fn))
    precisions.append(tp/(tp+fp))
    recalls.append(tp/(tp+fn))

    #By-event metrics
    gtAnnotations=labelVectorToAnnotations(testLabels,samplerate)   #<- or just filter the annotations
    f,r,p=annotationPairToMetrics(gtAnnotations,detections)

    #calculate metrics
    eventF1s.append(f)
    eventPrecisions.append(p)
    eventRecalls.append(r)

#include metrics in the dataframe
experimentModels_N2['rawF1']=rawF1s
experimentModels_N2['rawPrecision']=rawPrecisions
experimentModels_N2['rawRecall']=rawRecalls

experimentModels_N2['f1']=f1s
experimentModels_N2['precision']=precisions
experimentModels_N2['recall']=recalls

experimentModels_N2['eventF1']=eventF1s
experimentModels_N2['eventPrecision']=eventPrecisions
experimentModels_N2['eventRecall']=eventRecalls


*************************
1 of 15
*************************
2 of 15
*************************
3 of 15
*************************
4 of 15
*************************
5 of 15
*************************
6 of 15
*************************
7 of 15
*************************
8 of 15
*************************
9 of 15
*************************
10 of 15
*************************
11 of 15
*************************
12 of 15
*************************
13 of 15
*************************
14 of 15
*************************
15 of 15


In [38]:
dumpPickle('experimentModelsTest_N2_union_confidence_temp.pkl',experimentModels_N2)

In [39]:
fig=px.scatter(experimentModels_N2,x='rawF1',y='f1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels_N2['rawF1'], y=experimentModels_N2['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [40]:
fig=px.scatter(experimentModels_N2,x='rawF1',y='eventF1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels_N2['rawF1'], y=experimentModels_N2['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [41]:
fig=px.scatter(experimentModels_N2,x='eventF1',y='eventPrecision',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels_N2['eventF1'], y=experimentModels_N2['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [42]:
fig=px.scatter(experimentModels_N2,x='eventF1',y='eventRecall',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels_N2['eventF1'], y=experimentModels_N2['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [43]:
experimentModels_N2[['test','eventF1','eventPrecision','eventRecall']].groupby('test').describe(percentiles=[0.5])

eventF1                                             eventPrecision  \
       count      mean std       min       50%       max          count   
test                                                                      
0001     1.0  0.598616 NaN  0.598616  0.598616  0.598616            1.0   
0002     1.0  0.876666 NaN  0.876666  0.876666  0.876666            1.0   
0003     1.0  0.805219 NaN  0.805219  0.805219  0.805219            1.0   
0005     1.0  0.815275 NaN  0.815275  0.815275  0.815275            1.0   
0006     1.0  0.721455 NaN  0.721455  0.721455  0.721455            1.0   
0007     1.0  0.815692 NaN  0.815692  0.815692  0.815692            1.0   
0009     1.0  0.775087 NaN  0.775087  0.775087  0.775087            1.0   
0010     1.0  0.868465 NaN  0.868465  0.868465  0.868465            1.0   
0011     1.0  0.838381 NaN  0.838381  0.838381  0.838381            1.0   
0012     1.0  0.842305 NaN  0.842305  0.842305  0.842305            1.0   
0013     1.0  0.797309 NaN  0.797309  0.797309  0.797309            1.0   
0014     1.0  0.692660 NaN  0.692660  0.692660  0.692660            1.0   
0017     1.0  0.826573 NaN  0.826573  0.826573  0.826573            1.0   
0018     1.0  0.816311 NaN  0.816311  0.816311  0.816311            1.0   
0019     1.0  0.721091 NaN  0.721091  0.721091  0.721091            1.0   

                                                 eventRecall                \
          mean std       min       50%       max       count      mean std   
test                                                                         
0001  0.997118 NaN  0.997118  0.997118  0.997118         1.0  0.427689 NaN   
0002  0.852696 NaN  0.852696  0.852696  0.852696         1.0  0.902144 NaN   
0003  0.915433 NaN  0.915433  0.915433  0.915433         1.0  0.718333 NaN   
0005  0.735734 NaN  0.735734  0.735734  0.735734         1.0  0.913997 NaN   
0006  0.911765 NaN  0.911765  0.911765  0.911765         1.0  0.596871 NaN   
0007  0.721266 NaN  0.721266  0.721266  0.721266         1.0  0.938751 NaN   
0009  0.924590 NaN  0.924590  0.924590  0.924590         1.0  0.665868 NaN   
0010  0.953846 NaN  0.953846  0.953846  0.953846         1.0  0.796689 NaN   
0011  0.752289 NaN  0.752289  0.752289  0.752289         1.0  0.946549 NaN   
0012  0.788802 NaN  0.788802  0.788802  0.788802         1.0  0.903830 NaN   
0013  0.676871 NaN  0.676871  0.676871  0.676871         1.0  0.970035 NaN   
0014  0.532301 NaN  0.532301  0.532301  0.532301         1.0  0.987029 NaN   
0017  0.933684 NaN  0.933684  0.933684  0.933684         1.0  0.741423 NaN   
0018  0.723835 NaN  0.723835  0.723835  0.723835         1.0  0.935795 NaN   
0019  0.573001 NaN  0.573001  0.573001  0.573001         1.0  0.970703 NaN   

                                    
           min       50%       max  
test                                
0001  0.427689  0.427689  0.427689  
0002  0.902144  0.902144  0.902144  
0003  0.718333  0.718333  0.718333  
0005  0.913997  0.913997  0.913997  
0006  0.596871  0.596871  0.596871  
0007  0.938751  0.938751  0.938751  
0009  0.665868  0.665868  0.665868  
0010  0.796689  0.796689  0.796689  
0011  0.946549  0.946549  0.946549  
0012  0.903830  0.903830  0.903830  
0013  0.970035  0.970035  0.970035  
0014  0.987029  0.987029  0.987029  
0017  0.741423  0.741423  0.741423  
0018  0.935795  0.935795  0.935795  
0019  0.970703  0.970703  0.970703

In [44]:
experimentModels_N2[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean()

,test,eventF1,eventPrecision,eventRecall
0,0001,0.598616,0.997118,0.427689
1,0002,0.876666,0.852696,0.902144
2,0003,0.805219,0.915433,0.718333
3,0005,0.815275,0.735734,0.913997
4,0006,0.721455,0.911765,0.596871
5,0007,0.815692,0.721266,0.938751
6,0009,0.775087,0.924590,0.665868
7,0010,0.868465,0.953846,0.796689
8,0011,0.838381,0.752289,0.946549
9,0012,0.842305,0.788802,0.903830


In [45]:
experimentModels_N2[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().mean()

test              6.668000e+54
eventF1           7.874069e-01
eventPrecision    7.995488e-01
eventRecall       8.277138e-01
dtype: float64

In [46]:
experimentModels_N2[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().std()

C:\Users\Gurdiel\AppData\Local\Temp\ipykernel_9612\813125388.py:1: FutureWarning:

The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



eventF1           0.074656
eventPrecision    0.141773
eventRecall       0.165372
dtype: float64

In [47]:
auxPrecision=pd.DataFrame({
    'metric':'event precision',
    'value':experimentModels_N2.eventPrecision,
    'event F1':experimentModels_N2.eventF1
})

auxRecall=pd.DataFrame({
    'metric':'event recall',
    'value':experimentModels_N2.eventRecall,
    'event F1':experimentModels_N2.eventF1
})
visualTradeoff=pd.concat((auxPrecision,auxRecall))

In [48]:
fig=px.scatter(visualTradeoff,x='event F1',y='value',color='metric', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

### with RED test set

In [62]:
experimentModels_N2[experimentModels_N2.test.isin(['0002','0006','0012','0013'])][['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().mean()

test              5.001500e+11
eventF1           8.094335e-01
eventPrecision    8.075335e-01
eventRecall       8.432200e-01
dtype: float64

### remove outliers

In [56]:
np.setdiff1d(experimentModels_N2.test,['0001','0014','0019'])

array(['0002', '0003', '0005', '0006', '0007', '0009', '0010', '0011',
       '0012', '0013', '0017', '0018'], dtype=object)

In [63]:
experimentModels_N2[experimentModels_N2.test.isin(np.setdiff1d(experimentModels_N2.test,['0001','0014','0019','0006']))][['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().mean()

test              1.818455e+39
eventF1           8.252076e-01
eventPrecision    8.162769e-01
eventRecall       8.575831e-01
dtype: float64

In [54]:
def annotationPairToGraph(annotations,detections,thresIoU=0.2):
    #get the coords
    gtCoords=zip(annotations.startInd,annotations.stopInd)
    outCoords=zip(detections.startInd,detections.stopInd)
    #calculate the iou vector
    iouVector=np.array(list(itt.starmap(getIou,itt.product(gtCoords,outCoords))))
    #reshape to a matrix
    iouMatrix=iouVector.reshape(len(annotations),len(detections))
    #create tables
    index0=np.apply_along_axis(np.argmax,0,iouMatrix)
    iou0=np.apply_along_axis(np.max,0,iouMatrix)
    index1=np.apply_along_axis(np.argmax,1,iouMatrix)
    iou1=np.apply_along_axis(np.max,1,iouMatrix)
    
    tableOut=pd.DataFrame({
        'indexGT':index0,
        'iou':iou0
    })

    tableGT=pd.DataFrame({
        'indexOut':index1,
        'iou':iou1
    })

    tableOut['type']='out'
    tableGT['type']='gt'
    tableOut['indexOut']=tableOut.index
    tableGT['indexGT']=tableGT.index
    #Correct external indexes of objects not overlapping
    tableOut.loc[tableOut.iou==0,'indexGT']='NA'
    tableGT.loc[tableGT.iou==0,'indexOut']='NA'
    #set tps
    tableOut['tp']=tableOut.iou>thresIoU
    tableOut['fp']=tableOut.iou<=thresIoU
    tableGT['tp']=tableGT.iou>thresIoU
    tableGT['fn']=tableGT.iou<=thresIoU
    #calculate metrics
    recall=np.sum(tableGT['tp'])/len(tableGT)
    precision=np.sum(tableOut['tp'])/len(tableOut)
    f1=(np.sum(tableGT['tp'])+np.sum(tableOut['tp']))/(len(tableGT)+len(tableOut))
    #concatenate tables
    appended=pd.concat(objs=(tableOut,tableGT),axis=0)
    #modify values
    appended['x']=appended['indexGT']
    appended['y']=appended['indexOut']
    appended.loc[((appended.type=='out')&(~ appended.tp)),'x']=-10
    appended.loc[((appended.type=='gt')&(~ appended.tp)),'y']=-10
    appended['size']=1
    appended.loc[appended.type=='out','size']=3
    #create the graph
    minTPIoU=np.min(appended[appended.tp].iou)
    fig=px.scatter(appended,x='x',y='y',color='iou',symbol='type',
    opacity=0.8,symbol_map={'out':'circle-open','gt':'circle'},size='size',
    color_continuous_scale=
        ((0.0, 'rgb(40,40,40)'),
        (0.000001, 'rgb(28,227,255)'),
        (0.14, 'rgb(56,199,255)'),
        (0.29, 'rgb(85,170,255)'),
        (0.42, 'rgb(113,142,255)'),
        (0.57, 'rgb(142,113,255)'),
        (0.71, 'rgb(170,85,255)'),
        (0.86, 'rgb(199,56,255)'),
        (1.0, 'rgb(227,28,255)')),
    range_x=(-20,len(tableGT)+10),range_y=(-20,len(tableOut)+10),
    title='by-Event evaluation summary<br><sup>F1(@IoU>'+str(thresIoU)+')='+str(round(f1,4))+' | minimum TP IoU: '+str(round(minTPIoU,4))+'</sup>',
    hover_data={'x':False,
    'y':False,
    'tp':False,
    'fp':False,
    'fn':False,
    'size':False,
    'type':False,
    'iou':':.4f', # customize hover for column of y attribute
    'indexGT':True,
    'indexOut':True
    })
    for t in fig.data:
        t.marker.line.width = 2
    fig.update_xaxes(title_text=str(len(tableGT))+' ANNOTATIONS | recall(@IoU>'+str(thresIoU)+')= '+str(round(recall,4)))
    fig.update_yaxes(title_text=str(len(tableOut))+' DETECTIONS | precision(@IoU>'+str(thresIoU)+')= '+str(round(precision,4)))
    fig.add_vline(x=-5,line_dash='dash')
    fig.add_hline(y=-5,line_dash='dash')
    #----------------------------------------------------------------------->
    # https://stackoverflow.com/questions/61827165/plotly-how-to-handle-overlapping-colorbar-and-legends
    # @vestland answer
    """ fig.update_layout(coloraxis_colorbar=dict(yanchor="top", y=1, x=0,
                                            ticks="outside",
                                            ticksuffix=" bills")) """
    # @bitbang answer
    fig.update_layout(legend_orientation="h")
    #<----------------------------------------------------------------------
    return fig

In [55]:
annotationPairToGraph(gtAnnotations,processedAnnotations)

### represent a 0.8 spindle detection

In [65]:
experimentModels_N2.iloc[1]

criteriumId                                                     0003
criteriumName                                                  union
labelerIdList                                           [0001, 0002]
train              [0001, 0003, 0005, 0007, 0009, 0010, 0011, 001...
val                                               [0006, 0013, 0019]
test                                                            0002
modelId                         6491a608-d9e0-4af8-bfcb-4213b7921942
spindleTimeRate                                             0.062867
rawF1                                                       0.729162
rawPrecision                                                 0.84699
rawRecall                                                   0.640113
f1                                                          0.760092
precision                                                   0.844349
recall                                                      0.691124
eventF1                           

In [66]:
del row

In [88]:
#load model
model=loadBooster(experimentModels_N2.iloc[1].modelId,experimentId,datapath)

testSubjectId=experimentModels_N2.iloc[1].test
testHypnogram=hypnograms[testSubjectId]
testN2Mask=testHypnogram==2
#Define annotations criterium
usedAnnotations=annotationsN2[annotationsN2.labelerId.isin(experimentModels_N2.iloc[1].labelerIdList)].reset_index(drop=True)
#Load features and labels
testFeatures=loadFeatureMatrix([testSubjectId],featureSelection,signalsMetadata,samplerate,datapath)
testLabels=loadLabelsVector([testSubjectId],usedAnnotations,signalsMetadata,samplerate)

#Predict
testDMatrix=xgb.DMatrix(data=testFeatures)
probabilities=model.predict(testDMatrix,iteration_range=(0,hyperDepth))
rawLabels=probabilities>=hyperThres

#Apply mask ---------------------------------->
rawLabels=rawLabels*testN2Mask
#<---------------------------------------------

#Raw Metrics
rawTp=np.sum(rawLabels*testLabels)
rawFp=np.sum(rawLabels*(1-testLabels))
rawTn=np.sum((1-rawLabels)*(1-testLabels))
rawFn=np.sum((1-rawLabels)*testLabels)
#Raw appends
rawF1s.append(2*rawTp/(2*rawTp+rawFp+rawFn))
rawPrecisions.append(rawTp/(rawTp+rawFp) )
rawRecalls.append(rawTp/(rawTp+rawFn))
#Process
processedLabels=labelingProcess(rawLabels,hyperClose,hyperDuration,samplerate)
#Detections with confidence
detections=rawAndProcessedToConfidenceAnnotations(rawLabels,processedLabels,samplerate)
#Confidence threshold
detections=detections[detections.confidence>hyperConfidence].reset_index(drop=True)
#recalculate processed labels
processedLabels=excerptAnnotationsToLabels(detections,len(processedLabels))
#Processed metrics
tp=np.sum(processedLabels*testLabels)
fp=np.sum(processedLabels*(1-testLabels))
tn=np.sum((1-processedLabels)*(1-testLabels))
fn=np.sum((1-processedLabels)*testLabels)
#Processed appends
f1s.append(2*tp/(2*tp+fp+fn))
precisions.append(tp/(tp+fp))
recalls.append(tp/(tp+fn))

#By-event metrics
gtAnnotations=labelVectorToAnnotations(testLabels,samplerate)   #<- or just filter the annotations
f,r,p=annotationPairToMetrics(gtAnnotations,detections)

In [70]:
signals = loadMASSSpindles(masspath,returnSignals=True)[0]

In [72]:
signal=signals['0002']
signal=sg.resample_poly(signal,up=25,down=32)

In [73]:
len(signal)

7003200

In [74]:
len(processedLabels)

7003200

In [85]:
from plotly.subplots import make_subplots

roi=[8*10**3,8*10**5]

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(y=processedLabels[roi[0]:roi[1]]*1, fill='tozeroy',mode='none',name='output'))
fig.add_trace(go.Scatter(y=testLabels[roi[0]:roi[1]], fill='tozeroy',mode='none',name='GT'))
fig.add_trace(go.Scatter(y=signal[roi[0]:roi[1]],name='signal'), secondary_y=True)
fig.write_html("test 0002 N2.html")

In [89]:
N2_detections=detections.copy()

In [90]:
#load model
model=loadBooster(experimentModels_N2.iloc[1].modelId,experimentId,datapath)

testSubjectId=experimentModels_N2.iloc[1].test
testHypnogram=hypnograms[testSubjectId]
testN2Mask=testHypnogram==2
#Define annotations criterium
usedAnnotations=annotationsN2[annotationsN2.labelerId.isin(experimentModels_N2.iloc[1].labelerIdList)].reset_index(drop=True)
#Load features and labels
testFeatures=loadFeatureMatrix([testSubjectId],featureSelection,signalsMetadata,samplerate,datapath)
testLabels=loadLabelsVector([testSubjectId],usedAnnotations,signalsMetadata,samplerate)

#Predict
testDMatrix=xgb.DMatrix(data=testFeatures)
probabilities=model.predict(testDMatrix,iteration_range=(0,hyperDepth))
rawLabels=probabilities>=hyperThres

""" #Apply mask ---------------------------------->
rawLabels=rawLabels*testN2Mask
#<--------------------------------------------- """

#Raw Metrics
rawTp=np.sum(rawLabels*testLabels)
rawFp=np.sum(rawLabels*(1-testLabels))
rawTn=np.sum((1-rawLabels)*(1-testLabels))
rawFn=np.sum((1-rawLabels)*testLabels)
#Raw appends
rawF1s.append(2*rawTp/(2*rawTp+rawFp+rawFn))
rawPrecisions.append(rawTp/(rawTp+rawFp) )
rawRecalls.append(rawTp/(rawTp+rawFn))
#Process
processedLabels=labelingProcess(rawLabels,hyperClose,hyperDuration,samplerate)
#Detections with confidence
detections=rawAndProcessedToConfidenceAnnotations(rawLabels,processedLabels,samplerate)
#Confidence threshold
detections=detections[detections.confidence>hyperConfidence].reset_index(drop=True)
#recalculate processed labels
processedLabels=excerptAnnotationsToLabels(detections,len(processedLabels))
#Processed metrics
tp=np.sum(processedLabels*testLabels)
fp=np.sum(processedLabels*(1-testLabels))
tn=np.sum((1-processedLabels)*(1-testLabels))
fn=np.sum((1-processedLabels)*testLabels)
#Processed appends
f1s.append(2*tp/(2*tp+fp+fn))
precisions.append(tp/(tp+fp))
recalls.append(tp/(tp+fn))

#By-event metrics
gtAnnotations=labelVectorToAnnotations(testLabels,samplerate)   #<- or just filter the annotations
f,r,p=annotationPairToMetrics(gtAnnotations,detections)

In [87]:
roi=[8*10**3,8*10**5]

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(y=processedLabels[roi[0]:roi[1]]*1, fill='tozeroy',mode='none',name='output'))
fig.add_trace(go.Scatter(y=testLabels[roi[0]:roi[1]], fill='tozeroy',mode='none',name='GT'))
fig.add_trace(go.Scatter(y=signal[roi[0]:roi[1]],name='signal'), secondary_y=True)
fig.write_html("test 0002.html")

In [91]:
len(N2_detections)/len(detections)

0.9012248123271435